In [ ]:
import cv2
import numpy as np
import time
# A biblioteca picamera2 não é mais necessária

# --- MUDANÇA ESSENCIAL: ARQUIVO DE VÍDEO COMO INPUT ---
VIDEO_FILE_PATH = "video.mp4" 
# Por favor, substitua "video_para_analise.mp4" pelo caminho real do seu arquivo.
# ----------------------------------------------------

# --- PARÂMETROS DE DETECÇÃO (AJUSTE AQUI) ---
# Apenas os parâmetros de detecção de PRETO são mantidos.

LOWER_COLOR = np.array([95, 80, 150])
UPPER_COLOR = np.array([105, 255, 255])
# ----------------------------------------------------

# --- INICIALIZAÇÃO DO VÍDEO DE ENTRADA ---
print(f"Abrindo o vídeo de entrada: {VIDEO_FILE_PATH}")
cap = cv2.VideoCapture(VIDEO_FILE_PATH)

# Verificar se o arquivo de vídeo foi aberto com sucesso
if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o arquivo de vídeo em {VIDEO_FILE_PATH}")
    raise FileNotFoundError(f"Vídeo não encontrado ou formato inválido: {VIDEO_FILE_PATH}")

# 2. Configuração do Vídeo de Saída, baseada no INPUT
# Obtendo as propriedades do vídeo de entrada
largura = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
altura = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Se o FPS não for detectado ou for zero, usar um padrão (ex: 30)
if fps <= 0:
    fps = 30
    print(f"Aviso: FPS do vídeo de entrada não detectado, usando padrão de {fps}.")
    
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# Nome do arquivo de saída alterado para "saida_mascara.mp4"
out = cv2.VideoWriter("saida_mascara.mp4", fourcc, fps, (largura, altura))

print(f"Processando vídeo com dimensões: {largura}x{altura} a {fps:.2f} FPS.")

# Variável para guardar o último valor da área detectada
area_final = 0
frame_count = 0

# --- KERNEL PARA A LIMPEZA DA MÁSCARA ---
# Criamos um "elemento estruturante" para a limpeza da máscara.
kernel = np.ones((5,5), np.uint8)

# 3. Loop de Captura e Processamento
start_time = time.time()
while cap.isOpened():
    # cap.read() retorna:
    # ret (True/False): se o frame foi lido com sucesso
    # frame: o próprio frame (imagem BGR)
    ret, frame = cap.read() 

    if not ret:
        # Se ret for False, significa que o vídeo terminou ou houve um erro
        break
    
    frame_count += 1
    
    # Conversão do frame BGR (do vídeo) para HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- INÍCIO DA LÓGICA DE DETECÇÃO E LIMPEZA (SEM VALIDAÇÃO AZUL) ---
    
    # 1. Cria a máscara preta "suja" (granulada)
    mask_suja = cv2.inRange(hsv, LOWER_COLOR, UPPER_COLOR)

    # 2. Usa o "CLOSE" para preencher os buracos na máscara preta (limpeza)
    mask_limpa = cv2.morphologyEx(mask_suja, cv2.MORPH_CLOSE, kernel)

    # Encontrar contornos na máscara limpa
    contornos, _ = cv2.findContours(mask_limpa, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contornos:
        # Pegar maior contorno (a mancha preta)
        maior_contorno = max(contornos, key=cv2.contourArea)
        area_mancha = cv2.contourArea(maior_contorno)
        area_final = area_mancha 
    else:
        area_final = 0
    
    # --- FIM DA LÓGICA DE DETECÇÃO E LIMPEZA ---

    # 4. Salvar o frame da MÁSCARA LIMPA no vídeo de saída
    # Converte a máscara de 1 canal (Grayscale) para 3 canais (BGR) para escrita no vídeo
    mascara_colorida = cv2.cvtColor(mask_limpa, cv2.COLOR_GRAY2BGR)
    out.write(mascara_colorida)

# 5. Finalização
end_time = time.time()
elapsed_time = end_time - start_time
print("\nProcesso concluído!")
print(f"Frames processados: {frame_count}")
print(f"Tempo total de processamento: {elapsed_time:.2f} segundos")

# --- FINALIZAÇÃO DO CAPTURE E WRITER ---
cap.release() # Fecha o arquivo de vídeo de entrada
out.release() # Fecha o arquivo de vídeo de saída

# 6. Exibir os resultados finais
print(f"Vídeo da máscara salvo em: saida_mascara.mp4")
print(f"Área final da mancha preta no ÚLTIMO frame: {int(area_final)} pixels")


Abrindo o vídeo de entrada: video.mp4
Processando vídeo com dimensões: 1920x1080 a 30.00 FPS.

Processo concluído!
Frames processados: 3216
Tempo total de processamento: 64.64 segundos
Vídeo da máscara salvo em: saida_mascara.mp4
Área final da mancha preta no ÚLTIMO frame: 1013520 pixels
